## Clean up
Please make sure to comment the below section if you are planning to use the Knowledge Base that you created above for building your RAG application.
If you only wanted to try out creating the KB using SDK, then please make sure to delete all the resources that were created as you will be incurred cost for storing documents in OSS index.

#### Delete KnowledgeBase - uncomment and delete resources after completing all the notebooks.


In [3]:
import os
import boto3

profile_name = os.getenv("AWS_PROFILE", 'cloud-services-dev')
print(f"{profile_name=}")
region_name = 'us-west-2' # os.getenv("AWS_REGION", 'us-west-2')
print(f"{region_name=}")

boto3_session = boto3.Session(profile_name=profile_name, region_name=region_name)


bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)

profile_name='cloud-services-dev'
region_name='us-west-2'


In [4]:
%store -r kb_id
# kb_id = "<knowledge base id>" If you have already created knowledge base, comment the `store -r kb_id` and provide knowledge base id here.
kb_id

'35ZBLUQRA4'

In [5]:
# Create a DataSource in KnowledgeBase
ds = None
response = bedrock_agent_client.list_data_sources(knowledgeBaseId=kb_id)
if response.get('dataSourceSummaries', []):
    print(f"Data source exists")
    ds = response['dataSourceSummaries'][0]


Data source exists


In [6]:
aoss_client = boto3_session.client('opensearchserverless')


In [7]:
bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=kb_id)
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)


{'ResponseMetadata': {'RequestId': 'fbfaf332-848d-4a82-beb6-4c950383861a',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 05 Apr 2024 13:19:18 GMT',
   'content-type': 'application/json',
   'content-length': '52',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'fbfaf332-848d-4a82-beb6-4c950383861a',
   'x-amz-apigw-id': 'VwQNhE29vHcEO4Q=',
   'x-amzn-trace-id': 'Root=1-660ffa56-5551c6e50d19d9055b0f3bc6'},
  'RetryAttempts': 0},
 'knowledgeBaseId': '35ZBLUQRA4',
 'status': 'DELETING'}

In [17]:
import sys
import contextlib
with contextlib.suppress(KeyError):
    del sys.modules["utility"]
from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, set_names

# suffix = random.randrange(200, 900)
suffix = "724"
set_names(suffix)
vector_store_name = f'bedrock-sample-rag-{suffix}'
index_name = f"bedrock-sample-rag-index-{suffix}"


profile_name='cloud-services-dev'
region_name='us-west-2'
> set_names
encryption_policy_name='bedrock-sample-rag-sp-724'
network_policy_name='bedrock-sample-rag-np-724'
access_policy_name='bedrock-sample-rag-ap-724'
bedrock_execution_role_name='AmazonBedrockExecutionRoleForKnowledgeBase_724'
fm_policy_name='AmazonBedrockFoundationModelPolicyForKnowledgeBase_724'
s3_policy_name='AmazonBedrockS3PolicyForKnowledgeBase_724'
oss_policy_name='AmazonBedrockOSSPolicyForKnowledgeBase_724'
< set_names


In [11]:
response = aoss_client.list_collections(
    collectionFilters={
        'name': vector_store_name,
        # 'status': 'CREATING'|'DELETING'|'ACTIVE'|'FAILED'
    },
    maxResults=100,
)
if response.get('collectionSummaries', []):
    print(f"Collection {vector_store_name} already exists")
    collection = response['collectionSummaries'][0]


Collection bedrock-sample-rag-724 already exists


In [15]:
import pprint

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(collection)
collection_id = collection["id"]

{ 'arn': 'arn:aws:aoss:us-west-2:780733454460:collection/d48vuv5t2fz09fwnpv2f',
  'id': 'd48vuv5t2fz09fwnpv2f',
  'name': 'bedrock-sample-rag-724',
  'status': 'ACTIVE'}


In [23]:

sts_client = boto3_session.client('sts')
# boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'

s3_client = boto3_session.client('s3')
account_id = sts_client.get_caller_identity()["Account"]
s3_suffix = f"{region_name}-{account_id}"
bucket_name = f'bedrock-kb-{s3_suffix}'  # replace it with your bucket name.

bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Arn']

# create security, network and data access policies within OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(
    vector_store_name=vector_store_name,
    aoss_client=aoss_client,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn,
)


pp.pprint(access_policy)

pp.pprint(network_policy)

pp.pprint(encryption_policy)


> create_bedrock_execution_role
fm_policy_name='AmazonBedrockFoundationModelPolicyForKnowledgeBase_724'
Foundation model policy already exists
s3_policy_name='AmazonBedrockS3PolicyForKnowledgeBase_724'
S3 policy already exists
bedrock_execution_role_name='AmazonBedrockExecutionRoleForKnowledgeBase_724'
Bedrock execution role already exists
Encryption policy already exists
Network policy already exists
Data policy already exists
{ 'createdDate': 1712181403298,
  'lastModifiedDate': 1712181403298,
  'name': 'bedrock-sample-rag-ap-724',
  'policyVersion': 'MTcxMjE4MTQwMzI5OF8x',
  'type': 'data'}
{ 'createdDate': 1712181403123,
  'lastModifiedDate': 1712181546408,
  'name': 'bedrock-sample-rag-np-724',
  'policyVersion': 'MTcxMjE4MTU0NjQwOF8y',
  'type': 'network'}
{ 'createdDate': 1712181402948,
  'lastModifiedDate': 1712181402948,
  'name': 'bedrock-sample-rag-sp-724',
  'policyVersion': 'MTcxMjE4MTQwMjk0OF8x',
  'type': 'encryption'}


In [30]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

credentials = boto3_session.get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

service = 'aoss'
index_name = f"bedrock-sample-index-{suffix}"
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

# Delete index
if existing := oss_client.indices.exists(index=index_name):
    aoss_client.indices.delete(index=index_name)
    aoss_client.delete_collection(id=collection_id)
    aoss_client.delete_access_policy(type="data", name=access_policy['name'])
    aoss_client.delete_security_policy(type="network", name=network_policy['name'])
    aoss_client.delete_security_policy(type="encryption", name=encryption_policy['name'])

d48vuv5t2fz09fwnpv2f.us-west-2.aoss.amazonaws.com


AuthorizationException: AuthorizationException(403, '')

#### Delete role and policies


In [39]:
import sys
import contextlib
with contextlib.suppress(KeyError):
    del sys.modules["utility"]
from utility import delete_iam_role_and_policies
delete_iam_role_and_policies(account_number=account_id, suffix_p=suffix)

profile_name='cloud-services-dev'
region_name='us-west-2'
> set_names
encryption_policy_name='bedrock-sample-rag-sp-724'
network_policy_name='bedrock-sample-rag-np-724'
access_policy_name='bedrock-sample-rag-ap-724'
bedrock_execution_role_name='AmazonBedrockExecutionRoleForKnowledgeBase_724'
fm_policy_name='AmazonBedrockFoundationModelPolicyForKnowledgeBase_724'
s3_policy_name='AmazonBedrockS3PolicyForKnowledgeBase_724'
oss_policy_name='AmazonBedrockOSSPolicyForKnowledgeBase_724'
< set_names
s3_policy_arn='arn:aws:iam::780733454460:policy/AmazonBedrockS3PolicyForKnowledgeBase_724'
fm_policy_arn='arn:aws:iam::780733454460:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_724'
oss_policy_arn='arn:aws:iam::780733454460:policy/AmazonBedrockOSSPolicyForKnowledgeBase_724'
bedrock_execution_role_name='AmazonBedrockExecutionRoleForKnowledgeBase_724'
s3_policy_arn='arn:aws:iam::780733454460:policy/AmazonBedrockS3PolicyForKnowledgeBase_724'
fm_policy_arn='arn:aws:iam::780733454460:policy

0

#### Delete S3 objects

In [41]:
objects = s3_client.list_objects(Bucket=bucket_name)
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
s3_client.delete_bucket(Bucket=bucket_name)

NoSuchBucket: An error occurred (NoSuchBucket) when calling the ListObjects operation: The specified bucket does not exist